In [1]:
import os
weaviate_api_key = os.getenv("weaviate_api_key") or "weaviate_api_key"
weaviate_url = os.getenv("weaviate_url") or "weaviate_url"
cohere_api_key = os.getenv("COHERE_API_KEY") or "COHERE_API_KEY"

In [2]:
query = "How much inventory should I have?"

In [7]:
from langchain.llms import Cohere
llm = Cohere(model="d2cd24fd-67fd-4610-ab84-a37f9e635313-ft")

In [4]:
system_prompt = """You are an assistant specialized in ThruThink budgeting analysis and projection web application usage.
You are also knowledgeable in a wide range of budgeting and accounting topics, including EBITDA, cash flow balance, inventory management, and more.
While you strive to provide accurate information and assistance, please keep in mind that you are not a licensed investment advisor, financial advisor, or tax advisor.
Therefore, you cannot provide personalized investment advice, financial planning, or tax guidance.
You are here to offer general information, answer your questions, and assist with ThruThink-related inquiries to the best of your knowledge.
If you don't know the answer, just say that "I don't know", don't try to make up an answer."""

In [26]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

question_variation_prompt_template = """Please generate {varc} variations of the following prompt. Each variation should be in one line. Be creative. Don't include any preamble or explanations, just the variations.
Here is the prompt: {prompt}
Variations:"""

variation_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template(question_variation_prompt_template)
])

In [3]:
varc = 4

In [17]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

variation_system_prompt = """You are a helpful assistant that generates multiple search queries based on a single input query.
Each generated query should be in one line in a new line. Don't include any preamble, explanations, or don't answer the queries, just geenrate the queries."""
variation_user_command_prompt_template = "Generate multiple search queries related to: {query}"
variation_user_example_prompt_template = "Example output:"
for i in range(varc):
    variation_user_example_prompt_template += f" {i}. Query variation"

variation_user_output_prompt_template = "OUTPUT ({varc} numbered queries):"

variation_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(variation_system_prompt),
    HumanMessagePromptTemplate.from_template(variation_user_command_prompt_template),
    HumanMessagePromptTemplate.from_template(variation_user_example_prompt_template),
    HumanMessagePromptTemplate.from_template(variation_user_output_prompt_template)
])

In [20]:
query_variations.split("\n")[:varc]

[' 1. How much inventory should I have?',
 '2. How does inventory management software help with inventory management?',
 '3. How much inventory should I have at the end of the year?',
 '4. How much inventory should I have at the beginning of the year?']

In [29]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

variation_system_prompt = """Your task is to generate {varc} different search queries that aim to answer the user question from multiple perspectives.
The user questions are focused on ThruThink budgeting analysis and projection web application usage, or a wide range of budgeting and accounting topics, including EBITDA, cash flow balance, inventory management, and more.
Each query MUST tackle the question from a different viewpoint, we want to get a variety of RELEVANT search results.
Each query MUST be in one line and one line only. You SHOULD NOT include any preamble or explanations, and you SHOULD NOT answer the questions or add anything else, just geenrate the queries."""
variation_user_command_prompt_template = "Original question: {query}"
variation_user_example_prompt_template = "Example output:\n"
for i in range(varc):
    variation_user_example_prompt_template += f" {i + 1}. Query variation\n"

variation_user_output_prompt_template = "OUTPUT ({varc} numbered queries):"

variation_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(variation_system_prompt),
    HumanMessagePromptTemplate.from_template(variation_user_command_prompt_template),
    HumanMessagePromptTemplate.from_template(variation_user_example_prompt_template),
    HumanMessagePromptTemplate.from_template(variation_user_output_prompt_template)
])

In [30]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

variation_chain = (
    {
        "query": itemgetter("query"),
        "varc": itemgetter("varc")
    }
    | variation_prompt
    | llm
    | StrOutputParser()
)

In [31]:
query_variations = variation_chain.invoke({"query": query, "varc": varc})
query_variations

' 1. Is there a direct relationship between inventory and cash flow?\n2. Is there a direct relationship between inventory and cash flow balance?\n3. Is there a direct relationship between inventory and EBITDA?\n4. Is there a direct relationship between inventory and inventory management?'

In [32]:
queries = [query]
for query_variation in query_variations.split("\n")[:varc]:
    dot_index = query_variation.index(".")
    q = query_variation[dot_index + 1:].strip()
    if q not in queries:
        queries.append(q)

queries

['How much inventory should I have?',
 'Is there a direct relationship between inventory and cash flow?',
 'Is there a direct relationship between inventory and cash flow balance?',
 'Is there a direct relationship between inventory and EBITDA?',
 'Is there a direct relationship between inventory and inventory management?']

In [33]:
import weaviate

auth_config = weaviate.AuthApiKey(api_key=weaviate_api_key)

client = weaviate.Client(
  url=weaviate_url,
  auth_client_secret=auth_config,
  additional_headers={"X-Cohere-Api-Key": cohere_api_key}
)
client.is_ready()

True

In [34]:
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Weaviate

index_name = "Help"
text_key = "text"
vectorstore = Weaviate(client, index_name, text_key)
vectorstore._query_attrs = ["text", "title", "category", "slug", "_additional {distance}"]
vectorstore.embedding = CohereEmbeddings(model="embed-english-v2.0", cohere_api_key=cohere_api_key)

In [35]:
# Retrieve multi document sets
doc_count = 15
doc_sets = []
for q in queries:
    doc_sets.append(vectorstore.similarity_search_by_text(q, k=doc_count))

In [36]:
doc_hash = dict()
for doc_set in doc_sets:
    for rank, doc in enumerate(doc_set):
        title = doc.metadata['title']
        doc_hash[title] = True

In [37]:
doc_hash.keys()

dict_keys(['Asset Inputs', 'Inventory', 'Inventory Weeks to Hold', 'Accounts Receivable', 'Accrual Basis', 'Income Statement', 'Warrant Information', 'Cash-Beginning Working Capital', 'Accounts Receivable and Inventory Write Off', 'Cash Flow Control', 'Borrowing Base Repayment', 'EBITDA Multiple', 'EBITDA Projection Inputs', 'EBITDA Debt Multiples', 'EBITDA Definition'])

In [38]:
# Fuse + rerank document sets
rerank_k = 60
fused_scores = dict()
doc_map = dict()
for doc_set in doc_sets:
    for rank, doc in enumerate(doc_set):
        title = doc.metadata['title']
        if not title in doc_map:
            doc_map[title] = doc

        if title not in fused_scores:
            fused_scores[title] = 0

        fused_scores[title] += 1 / (rank + rerank_k)

reranked_results = [
    (doc_map[title], score)
    for title, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
]

In [39]:
[r[0].metadata["title"] for r in reranked_results]

['Inventory',
 'EBITDA Multiple',
 'Inventory Weeks to Hold',
 'Accounts Receivable',
 'Accrual Basis',
 'Income Statement',
 'EBITDA Definition',
 'Warrant Information',
 'Cash-Beginning Working Capital',
 'Asset Inputs',
 'EBITDA Debt Multiples',
 'EBITDA Projection Inputs',
 'Cash Flow Control',
 'Borrowing Base Repayment',
 'Accounts Receivable and Inventory Write Off']

In [40]:
def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [42]:
import cohere
co = cohere.Client(cohere_api_key)

In [45]:
top_k_doc = 5
documents_to_cohere_rank = []
for rrr in reranked_results:
    documents_to_cohere_rank.append(rrr[0].page_content)

cohere_ranks = co.rerank(
    query=query,
    documents=documents_to_cohere_rank,
    max_chunks_per_doc=100,
    top_n=top_k_doc,
    model="rerank-english-v2.0"
)

In [54]:
len(cohere_ranks)

TypeError: object of type 'Reranking' has no len()

In [50]:
[crr for crr in cohere_ranks]

[RerankResult<document['text']: measure of a Company's true performance from its operations., index: 1, relevance_score: 0.4672406>,
 RerankResult<document['text']: # Inventory
 ## Category: Inventory
 Inventory is unused material and supplies used to create finished product as well as unsold finished product.
 Inventory includes Work in Process which will include allocated manufacturing labor as well as the direct materials and supplies of unfinished Finished Product.
 Unsold completed Finished Product Inventory includes allocated direct manufacturing labor as well as the direct materials and supplies.
 Cash paid to create inventory (see Inventory - Future Terms Set-up) or User capitalized expenses (see Info Browser - Individually Scheduled Income/Expense, Capitalized Expenses) is counted as an Inventory Asset and not counted as an Expense in either the ThruThink Accrual or Cash Basis method of accounting (see Info Browser - Tax Rates). Therefore when the User chooses the ThruThink Ca

In [52]:
context = ""
documents = []
for index, crr in enumerate(cohere_ranks):
    if context:
        context += "\n"

    rrr = reranked_results[crr.index]
    context_content = rrr[0].page_content  # .replace("\n", " ")
    context += f"{index + 1}. context: `{context_content}`"
    documents.append(dict(
        id=rrr[0].metadata["slug"],
        title=rrr[0].metadata["title"],
        category=rrr[0].metadata["category"],
        snippet=rrr[0].page_content,
    ))

In [53]:
context

"1. context: `measure of a Company's true performance from its operations.`\n2. context: `# Inventory\n## Category: Inventory\nInventory is unused material and supplies used to create finished product as well as unsold finished product.\nInventory includes Work in Process which will include allocated manufacturing labor as well as the direct materials and supplies of unfinished Finished Product.\nUnsold completed Finished Product Inventory includes allocated direct manufacturing labor as well as the direct materials and supplies.\nCash paid to create inventory (see Inventory - Future Terms Set-up) or User capitalized expenses (see Info Browser - Individually Scheduled Income/Expense, Capitalized Expenses) is counted as an Inventory Asset and not counted as an Expense in either the ThruThink Accrual or Cash Basis method of accounting (see Info Browser - Tax Rates). Therefore when the User chooses the ThruThink Cash Basis for calculation of income taxes, cash paid for inventory will not 

In [45]:
chat_system_prompt = """You are an assistant specialized in ThruThink budgeting analysis and projection web application usage.
You are also knowledgeable in a wide range of budgeting and accounting topics, including EBITDA, cash flow balance, inventory management, and more.
While you strive to provide accurate information and assistance, please keep in mind that you are not a licensed investment advisor, financial advisor, or tax advisor.
Therefore, you cannot provide personalized investment advice, financial planning, or tax guidance.
You are here to assist with ThruThink-related inquiries, or offer general information, answer questions to the best of your knowledge.
When provided, factor in any pieces of retrieved context to answer the question. Also factor in any
If you don't know the answer, just say that "I don't know", don't try to make up an answer."""

f"""Use the following pieces of retrieved context to answer the question.
---
Contexts: {context}
---
Question: {query}
Answer:
"""

# cohere_model = "d2cd24fd-67fd-4610-ab84-a37f9e635313-ft"
cohere_model = "command-nightly"
response = co.chat(
    model=cohere_model,
    prompt_truncation='auto',
    temperature=0.0,
    connectors=[{"id": "web-search"}],
    citation_quality="accurate",
    # documents=documents,
    # conversation_id="GUID",
    preamble_override=chat_system_prompt,
    message=query,
)

In [10]:
response

cohere.Chat {
	id: 14039cb7-ef0b-43fa-896d-bc3bad5b8929
	response_id: 14039cb7-ef0b-43fa-896d-bc3bad5b8929
	generation_id: aca1c93c-0baf-4708-a22f-9f8565cc902f
	message: How much inventory should I have?
	text: Businesses should always hold enough stock to meet unexpected spikes in consumer demand but not too much to cause overstock issues. The amount of inventory a business should hold depends on various factors such as customer demand, lead time to process orders, holding costs, and order costs. 

For example, if a business has a lead time of four weeks to process customer orders and their customers place orders for 100 pieces of a product twice a week, they should have enough inventory to cover 200 pieces to account for the lead time and expected orders. 

Additionally, businesses may choose to hold more inventory during peak seasons or periods, such as summer or holidays like Christmas, to account for increased customer demand. 

It's important to note that the appropriate inventor

In [46]:
chat_system_prompt = """You are an assistant specialized in ThruThink budgeting analysis and projection web application usage.
You are also knowledgeable in a wide range of budgeting and accounting topics, including EBITDA, cash flow balance, inventory management, and more.
While you strive to provide accurate information and assistance, please keep in mind that you are not a licensed investment advisor, financial advisor, or tax advisor.
Therefore, you cannot provide personalized investment advice, financial planning, or tax guidance.
You are here to assist with ThruThink-related inquiries, or offer general information, answer questions to the best of your knowledge.
When provided, factor in any pieces of retrieved context to answer the question.
If you don't know the answer, just say that "I don't know", don't try to make up an answer."""

f"""Use the following pieces of retrieved context to answer the question.
Contexts: {context}
Question: {query}
Answer:
"""

# cohere_model = "d2cd24fd-67fd-4610-ab84-a37f9e635313-ft"
cohere_model = "command-nightly"
response = co.chat(
    model=cohere_model,
    prompt_truncation='auto',
    temperature=0.0,
    # connectors=[{"id": "web-search"}],
    citation_quality="accurate",
    documents=documents,
    # conversation_id="GUID",
    preamble_override=chat_system_prompt,
    message=query,
)
response

cohere.Chat {
	id: f72b0ce3-9c3d-4719-9c4d-52b9391465f7
	response_id: f72b0ce3-9c3d-4719-9c4d-52b9391465f7
	generation_id: c897a507-3ac4-4cab-9822-4718910ca121
	message: How much inventory should I have?
	text: Inventory is essentially unused materials and supplies that help in creating a finished product or unsold finished products. 

To determine the amount of inventory you should maintain, you'll need to consider the average amount of time you plan to hold inventory for use. This metric is known as "Inventory Weeks to Hold". You can input this as a whole number representing the number of weeks. 

If you're entering beginning inventory amounts, this will help to record the beginning inventory for the end of the period just prior to Year 1 (the Effective Date). Additionally, you'll be required to enter the Sale Value of the inventory being used, which represents the value at which the inventory will be sold.
	conversation_id: None
	prompt: None
	chat_history: None
	preamble: None
	cli